In [1]:
import selenium
import re
import pandas as pd
import numpy as np
import time
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC 
from selenium.webdriver.common.action_chains import ActionChains
from cmath import nan
from geopy.geocoders import Nominatim

def initialize():
    #CHROMEDRIVER_PATH = '/opt/WebDriver/bin'
    WINDOW_SIZE = "1000,1024"


    chrome_options = Options()
    #chrome_options.add_argument("--headless")
    chrome_options.add_argument("--window-size=%s" % WINDOW_SIZE)
    #chrome_options.binary_location = CHROME_PATH

    # Creating a chromedriver instance
    driver = webdriver.Chrome('D:\DATA\DV\chromedriver.exe')#executable_path=CHROMEDRIVER_PATH, 

    driver.get("https://www.boliga.dk/resultat?zipCodes=2770")
    driver.implicitly_wait(2)#It instructs the webdriver to wait for a certain amount of time before elements load on the webpage.

    driver.find_element(By.XPATH, "//button[contains(text(), 'Tillad alle')]").click()

    return driver

driver = initialize()
    
def zip_cph():
    zip_df = pd.read_csv('zip_cph.csv', header=None)
    postal_areas = []

    for i in zip_df[1]:
        numbers = i.replace("'", "")
        numbers = numbers.replace(",", "")
        numbers = numbers.split()
        for number in numbers:
            number = number.replace("[", "")
            number = number.replace("]", "")
            postal_areas.append(number)
    return postal_areas

def zip_cph_extended():
    zip_df = pd.read_csv('zip_cph_extended.csv', header=None)

    postal_areas = []

    for i in zip_df[1]:
        numbers = i.replace("'", "")
        numbers = numbers.replace(",", "")
        numbers = numbers.split()
        for number in numbers:
            number = number.replace("[", "")
            number = number.replace("]", "")
            postal_areas.append(number)
    return postal_areas


def searchzip(postal_areas):
    searchzip = []
    for area in postal_areas:
        a = ("https://www.boliga.dk/resultat?zipCodes=" + area)
        print(a)
        searchzip.append(a)
    return searchzip

def get_houselinks(searchzip):
    proceed = True
    houselinks = []

    for search in searchzip:
        time.sleep(1)
        driver.get(search)
        proceed = True

        while proceed:
            time.sleep(1)
            houses = driver.find_elements(By.XPATH, "//a[@class = 'house-list-item']")
            for house in houses:
                houselink = house.get_attribute("href")
                houselinks.append(houselink)
            try:
                nextbutton = driver.find_element(By.XPATH, "//a[not(contains(@class, 'disabled')) and contains(text(), 'Næste')]")
                nextbutton.click()
            except:
                proceed = False
    return houselinks

def create_dataframe():
    df = pd.DataFrame(columns = ['Type', 'Address', 'Price', 'Sqm Price', 'Size', 'Living Area', 'Ground Area', 'Rooms', 'Ownership Expense', 'Year Built', 'Story', 'Energy Label', 'Description'])
    return df

def scrape(houselinks, df):
    for link in houselinks:
        try:
            driver.get(link)
            driver.implicitly_wait(2)

            try:
                house_address = driver.find_element(By.XPATH, "//*[@class = 'house-place ng-star-inserted']").text
            except: 
                house_address = "nan"

            try:
                type = driver.find_element(By.XPATH, "//span[@class = 'house-name']").text
            except: 
                type = "nan"

            try:
                house_price = driver.find_element(By.XPATH, "//*[@class = 'text-right house-price d-flex align-items-center justify-content-end ng-star-inserted']").text
            except: 
                house_price = "nan" 

            try:   
                price_sqm = driver.find_element(By.XPATH, "//*[@class = 'd-block text-right text-muted house-sqm-price ng-star-inserted']").text
            except: 
                price_sqm = "nan"

            try:
                house_size = driver.find_element(By.XPATH, "//span[contains(text(), 'Boligstørrelse:')]").text
            except: 
                house_size = "nan"

            try:
                living_area = driver.find_element(By.XPATH, "//span[contains(text(), 'Boligareal, tinglyst:')]").text
            except: 
                living_area = "nan"

            try:
                ground_area = driver.find_element(By.XPATH, "//span[contains(text(), 'Grundstørrelse:')]").text
            except: 
                ground_area = "nan"

            try:
                rooms = driver.find_element(By.XPATH, "//span[contains(text(), 'Værelser: ')]").text
            except: 
                rooms = "nan"

            try:
                ownership_expense = driver.find_element(By.XPATH, "//span[contains(text(), 'Ejerudgift:')]").text
            except: 
                ownership_expense = "nan"

            try:
                see_more_button = driver.find_element(By.XPATH, "//div[contains(text(), 'Vis mere')]")
                driver.execute_script('arguments[0].click()', see_more_button)
            except:
                pass

            try:
                year_built = driver.find_element(By.XPATH, "//span[contains(text(), 'Byggeår:')]").text
            except: 
                year_built = "nan"

            try:
                story = driver.find_element(By.XPATH, "//span[contains(text(), 'Sal:')]").text
            except:
                story = "st"

            try:
                energy_label = driver.find_element(By.XPATH, "//span[contains(text(), 'Energimærke:')]").text
            except:
                energy_label = "nan"  

            try:
                description = driver.find_element(By.XPATH, "//div[contains(@class, 'shared-property-information')]").text
            except:
                description = "nan"  

            df = df.append({'Type': type, 'Address' : house_address, 'Price' : house_price, 'Sqm Price' : price_sqm, 'Size': house_size, 'Living Area' : living_area, 'Ground Area': ground_area, 'Rooms': rooms, 'Ownership Expense': ownership_expense, 'Year Built': year_built, 'Story': story, 'Energy Label': energy_label, 'Description': description}, 
                        ignore_index = True) 
        except:
            pass
        
    return df

def get_coordinates(df):
    geolocator = Nominatim(user_agent="my_user_agent")
    addresses = df['Address'].values
    country ="Dk"
    longitude = []
    latitude=[]
    cleaned_address = []

    for address in addresses:
        address = re.sub("(?i)\d+\.+\s(th.|tv.|st.|mf.|th,|tv,|st,|mf,|th|tv|st|mf)", "", address)
        address = re.sub("\d+\s(th.|tv.|st.|mf.|th,|tv,|st,|mf,|th|tv|st|mf)", "", address)
        address = re.sub("\s(th.|tv.|st.|mf.|th,|tv,|st,|mf,|th|tv|st|mf)", "", address)
        address = address.replace(",", "")
        address = address.replace(".", "")
        print(address)
        cleaned_address.append(address)
        try:
            loc = geolocator.geocode(address)
            if loc:
                print(address)
                print(loc)
                print("latitude is :-" ,loc.latitude,"\nlongtitude is:-" ,loc.longitude)
                longitude.append(loc.longitude)
                latitude.append(loc.latitude)
            else:
                print(address)
                print("not found")
                longitude.append("nan")
                latitude.append("nan")
        except:
            print(address)
            print("not found")
            longitude.append("nan")
            latitude.append("nan")

    df["Longitude"] = longitude
    df["Latitude"] = latitude
    df["Cleaned_address"] = cleaned_address
    return df

def save_df(df, name):
    df.to_csv(name+'.csv') 

def append_df(df, df2):
   data = df.append(df2, ignore_index = True)
   return data  


def run_script():

    zips = searchzip(zip_cph())

    #zips = searchzip(zip_cph_extended()) uncomment if the scraper should be ran for the outskirts

    houselinks = get_houselinks(zips)

    df = create_dataframe()

    scraped_df = scrape(houselinks, df)

    df = get_coordinates(scraped_df)

    save_df(df, "scraped_data_boliga") # provide the name on how to save the .csv

    return df

run_script()


C:\Users\kovac\AppData\Local\Temp\ipykernel_11984\3931616264.py:28: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('D:\DATA\DV\chromedriver.exe')#executable_path=CHROMEDRIVER_PATH,


https://www.boliga.dk/resultat?zipCodes=2200


C:\Users\kovac\AppData\Local\Temp\ipykernel_11984\3931616264.py:181: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'Type': type, 'Address' : house_address, 'Price' : house_price, 'Sqm Price' : price_sqm, 'Size': house_size, 'Living Area' : living_area, 'Ground Area': ground_area, 'Rooms': rooms, 'Ownership Expense': ownership_expense, 'Year Built': year_built, 'Story': story, 'Energy Label': energy_label, 'Description': description},
C:\Users\kovac\AppData\Local\Temp\ipykernel_11984\3931616264.py:181: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'Type': type, 'Address' : house_address, 'Price' : house_price, 'Sqm Price' : price_sqm, 'Size': house_size, 'Living Area' : living_area, 'Ground Area': ground_area, 'Rooms': rooms, 'Ownership Expense': ownership_expense, 'Year Built': y

Bjelkes Allé 12 2 2 2200 København N
Bjelkes Allé 12 2 2 2200 København N
not found
Bjelkes Allé 34 1 30 2200 København N
Bjelkes Allé 34 1 30 2200 København N
not found
Heimdalsgade 14 2 5 2200 København N
Heimdalsgade 14 2 5 2200 København N
not found
Julius Bloms Gade 9 5 7 2200 København N
Julius Bloms Gade 9 5 7 2200 København N
not found
Stefansgade 10  2200 København N
Stefansgade 10  2200 København N
10, Stefansgade, Nørrebro, København, Københavns Kommune, Region Hovedstaden, 2200, Danmark
latitude is :- 55.696054 
longtitude is:- 12.544256
Frejasgade 15  2200 København N
Frejasgade 15  2200 København N
15, Frejasgade, Nørrebro, København, Københavns Kommune, Region Hovedstaden, 2200, Danmark
latitude is :- 55.698734 
longtitude is:- 12.551251
Ahornsgade 21  2200 København N
Ahornsgade 21  2200 København N
21, Ahornsgade, Nørrebro, København, Københavns Kommune, Region Hovedstaden, 2200, Danmark
latitude is :- 55.691647 
longtitude is:- 12.559237
Nørrebrogade 12  2200 Københav

,Type,Address,Price,Sqm Price,Size,Living Area,Ground Area,Rooms,Ownership Expense,Year Built,Story,Energy Label,Description,Longitude,Latitude,Cleaned_address
0,Ejerlejlighed,"Bjelkes Allé 12, 2. 2., 2200 København N",2.695.000 kr.,49.907 kr. / m²,Boligstørrelse: 54 m²,"Boligareal, tinglyst: 45 m²",nan,nan,Ejerudgift: 2.495 kr. / md.,nan,Sal: 2.,Energimærke: C,"Bjelkes Allé 12, 2. 2., 2200 København N\nEjer...",nan,nan,Bjelkes Allé 12 2 2 2200 København N
1,Ejerlejlighed,"Bjelkes Allé 34, 1. 30, 2200 København N",2.695.000 kr.,57.340 kr. / m²,Boligstørrelse: 47 m²,"Boligareal, tinglyst: 39 m²",nan,Værelser: 2,Ejerudgift: 2.434 kr. / md.,Byggeår: 1969,Sal: 1.,Energimærke: C,"Bjelkes Allé 34, 1. 30, 2200 København N\nEjer...",nan,nan,Bjelkes Allé 34 1 30 2200 København N
2,Ejerlejlighed,"Heimdalsgade 14, 2. 5, 2200 København N",5.495.000 kr.,54.950 kr. / m²,Boligstørrelse: 100 m²,"Boligareal, tinglyst: 85 m²",nan,Værelser: 3,Ejerudgift: 3.077 kr. / md.,Byggeår: 1939,Sal: 2.,Energimærke: C,"Heimdalsgade 14, 2. 5, 2200 København N\nEjerl...",nan,nan,Heimdalsgade 14 2 5 2200 København N
3,Ejerlejlighed,"Julius Bloms Gade 9, 5. 7, 2200 København N",1.995.000 kr.,44.333 kr. / m²,Boligstørrelse: 45 m²,"Boligareal, tinglyst: 38 m²",nan,Værelser: 2,Ejerudgift: 1.626 kr. / md.,Byggeår: 1907,Sal: 5.,Energimærke: D,"Julius Bloms Gade 9, 5. 7, 2200 København N\nE...",nan,nan,Julius Bloms Gade 9 5 7 2200 København N
4,Ejerlejlighed,"Stefansgade 10, 5. th, 2200 København N",2.295.000 kr.,60.394 kr. / m²,Boligstørrelse: 38 m²,"Boligareal, tinglyst: 31 m²",nan,Værelser: 2,Ejerudgift: 1.250 kr. / md.,Byggeår: 1886,Sal: 5.,Energimærke: C,"Stefansgade 10, 5. th, 2200 København N\nEjerl...",12.544256,55.696054,Stefansgade 10 2200 København N
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
112,Ejerlejlighed,"Bregnerødgade 21, 1. th., 2200 København N",-12%\n3.695.000 kr.,49.266 kr. / m²,Boligstørrelse: 75 m²,"Boligareal, tinglyst: 67 m²",nan,Værelser: 3,Ejerudgift: 4.218 kr. / md.,Byggeår: 1930,Sal: 1.,Energimærke: D,"Bregnerødgade 21, 1. th., 2200 København N\nEj...",12.537568,55.698183,Bregnerødgade 21 2200 København N
113,Ejerlejlighed,"Sortedam Dossering 41A, 3. tv., 2200 København N",-11%\n6.250.000 kr.,56.306 kr. / m²,Boligstørrelse: 111 m²,"Boligareal, tinglyst: 101 m²",nan,Værelser: 4,Ejerudgift: 6.522 kr. / md.,Byggeår: 1963,Sal: 3.,Energimærke: D,"Sortedam Dossering 41A, 3. tv., 2200 København...",nan,nan,Sortedam Dossering 41A 2200 København N
114,Ejerlejlighed,"Guldbergsgade 5, 5. 3., 2200 København N",-7%\n2.798.000 kr.,62.177 kr. / m²,Boligstørrelse: 45 m²,"Boligareal, tinglyst: 40 m²",nan,Værelser: 2,Ejerudgift: 1.802 kr. / md.,Byggeår: 1893,Sal: 5.,Energimærke: D,"Guldbergsgade 5, 5. 3., 2200 København N\nEjer...",nan,nan,Guldbergsgade 5 5 3 2200 København N
115,Ejerlejlighed,"Schleppegrellsgade 6, kl. th., 2200 København N",4.295.000 kr.,48.806 kr. / m²,Boligstørrelse: 88 m²,"Boligareal, tinglyst: 79 m²",nan,Værelser: 3,Ejerudgift: 2.358 kr. / md.,Byggeår: 1892,st,Energimærke: C,"Schleppegrellsgade 6, kl. th., 2200 København ...",nan,nan,Schleppegrellsgade 6 kl 2200 København N
